In [1]:
# import needed libraries

import csv
import json
import os

import requests as rq
from bs4 import BeautifulSoup as bs

import nltk

from transformers import BertTokenizer, BertModel
import torch

c:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Download the needed nltk models

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [17]:
# import nltk needed models

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [34]:
# scrape and save the data into "rugs.json" and into "rugs-stemmed-description.csv" with stemmer

request = rq.get(url='https://www.babakech.com/kilim-moroccan-rugs?page=3')
if request.status_code == 200:
    page = bs(request.content, 'html.parser')
    items = page.find_all(class_='product_details')

    for i in items:
        if i.find(class_='product-tag').text[:5].strip() == 'SKU:':
            product_url = i.find(class_='product-item-link').get('href')
            product_full_url = 'https://www.babakech.com' + product_url
            

            # url = 'https://www.babakech.com/vintage-rugs/colorful-beni-ouarian-1334.html'
            req = rq.get(url=product_full_url)
            if req.status_code == 200:
                soup = bs(req.content, 'html.parser')
                img = soup.find(class_='zoneimage fancybox').get('href')
                title = soup.find(class_='product_title').text[:-1].lower()

                price = soup.find(id='price-amount').text
                component = soup.find(class_='product_short_description')
                descriptions = component.find_all('p')
                try:
                    if descriptions[3].text[1:12] == 'Materials :':
                        desc = descriptions[0].text.lower()
                        craft = descriptions[2].text.lower()
                        reg = descriptions[4].text[9:].lower()
                        tri = descriptions[5].text[8:].lower()
                        
                        materials = descriptions[3].text[13:]
                        if materials.count(',') > 0:
                            materials_list = materials.split(',')
                            mat = [str(item.strip()).lower() for item in materials_list]
                        else:
                            print('\n\nthere is just one material\n')
                            mat = materials.strip().lower()
                        
                        # Tokenize the description
                        description = word_tokenize(desc)
                        # Remove stopwords
                        stop_words = set(stopwords.words('english'))
                        description = [word for word in description if word not in stop_words]
                        # Perform stemming
                        stemmer = PorterStemmer()
                        description = [stemmer.stem(word) for word in description]
                        
                        with open("rugs-stemmed-description.csv", mode='a', newline='') as f:
                            writer = csv.writer(f)
                            if materials.count(',') > 0:
                                writer.writerow([' '.join(description) + ' ' + ' '.join(mat) + ' ' + reg + ' ' + tri])
                            else:
                                writer.writerow([' '.join(description) + ' ' + mat + ' ' + reg + ' ' + tri])

                        d_json = {'image': str(img), 'title': str(title), 'price': int(price), 'description': str(desc), 'craft': str(craft), 'materials': mat, 'region': str(reg), 'tribe': str(tri)}
                        with open('rugs.json', 'rb+') as f:
                            f.seek(-1, os.SEEK_END)
                            f.truncate()
                            f.write(b',\n')
                            new_d_json = json.dumps(d_json, indent=4)
                            f.write(bytes(new_d_json, 'utf-8'))
                            f.write(b'\n]')

                    elif descriptions[2].text[1:12] == 'Materials :':
                        craft = descriptions[1].text.lower()
                        reg = descriptions[3].text[9:].lower()
                        tri = descriptions[4].text[8:].lower()
                        
                        materials = descriptions[2].text[13:]
                        if materials.count(',') > 0:
                            materials_list = materials.split(',')
                            mat = [str(item.strip()).lower() for item in materials_list]
                        else:
                            print('\n\nthere is just one material\n')
                            mat = materials.strip().lower()
                        
                        with open("rugs-stemmed-description.csv", mode='a', newline='') as f:
                            writer = csv.writer(f)
                            if materials.count(',') > 0:
                                writer.writerow([' '.join(mat) + ' ' + reg + ' ' + tri])
                            else:
                                writer.writerow([mat + ' ' + reg + ' ' + tri])

                        d_json = {'image': str(img), 'title': str(title), 'price': int(price), 'description': None, 'craft': str(craft), 'materials': mat, 'region': str(reg), 'tribe': str(tri)}
                        with open('rugs.json', 'rb+') as f:
                            f.seek(-1, os.SEEK_END)
                            f.truncate()
                            f.write(b',\n')
                            new_d_json = json.dumps(d_json, indent=4)
                            f.write(bytes(new_d_json, 'utf-8'))
                            f.write(b'\n]')
                    
                    else:
                        print('\n\nNo material found!!\n')

                except:
                    print(f"extraction of '{product_full_url}' data faild")

In [18]:
# scrape and save the data into "rugs.json" and into "rugs-lemmatized-description.csv" with lemmatizer

# https://www.babakech.com/vintage-rugs/
# https://www.babakech.com/checkered-rugs/
# https://www.babakech.com/abstract-moroccan-rugs/
# https://www.babakech.com/abstract-moroccan-rugs?page=2
# https://www.babakech.com/abstract-moroccan-rugs?page=3
# https://www.babakech.com/abstract-moroccan-rugs?page=4
# https://www.babakech.com/kilim-moroccan-rugs/
# https://www.babakech.com/kilim-moroccan-rugs?page=2
# https://www.babakech.com/kilim-moroccan-rugs?page=3

request = rq.get(url='https://www.babakech.com/kilim-moroccan-rugs?page=3')
if request.status_code == 200:
    page = bs(request.content, 'html.parser')
    items = page.find_all(class_='product_details')

    for i in items:
        if i.find(class_='product-tag').text[:5].strip() == 'SKU:':
            product_url = i.find(class_='product-item-link').get('href')
            product_full_url = 'https://www.babakech.com' + product_url
            

            req = rq.get(url=product_full_url)
            if req.status_code == 200:
                soup = bs(req.content, 'html.parser')
                img = soup.find(class_='zoneimage fancybox').get('href')
                title = soup.find(class_='product_title').text[:-1].lower()

                price = soup.find(id='price-amount').text
                component = soup.find(class_='product_short_description')
                descriptions = component.find_all('p')
                try:
                    if descriptions[3].text[1:12] == 'Materials :':
                        desc = descriptions[0].text.lower().replace('\u00a0', ' ').replace('\u00e9', 'e').replace('\u2019', '\'').replace('\u2014', '-')
                        craft = descriptions[2].text.lower()
                        reg = descriptions[4].text[9:].lower()
                        tri = descriptions[5].text[8:].lower()
                        
                        materials = descriptions[3].text[13:]
                        if materials.count(',') > 0:
                            materials_list = materials.split(',')
                            mat = [str(item.strip()).lower() for item in materials_list]
                        else:
                            print('\n\nthere is just one material\n')
                            mat = materials.strip().lower()
                        
                        # Tokenize the description
                        description = word_tokenize(desc)
                        # Remove stopwords
                        stop_words = set(stopwords.words('english'))
                        description = [word for word in description if word not in stop_words]
                        # Perform lemmatization
                        lemmatizer = WordNetLemmatizer()
                        words = [lemmatizer.lemmatize(word) for word in description]
                        
                        with open("rugs-lemmatized-description.csv", mode='a', newline='') as f:
                            writer = csv.writer(f)
                            if materials.count(',') > 0:
                                writer.writerow([' '.join(description) + ' ' + ' '.join(mat) + ' ' + reg + ' ' + tri])
                            else:
                                writer.writerow([' '.join(description) + ' ' + mat + ' ' + reg + ' ' + tri])

                        d_json = {'image': str(img), 'title': str(title), 'price': int(price), 'description': str(desc), 'craft': str(craft), 'materials': mat, 'region': str(reg), 'tribe': str(tri)}
                        with open('rugs.json', 'rb+') as f:
                            f.seek(-1, os.SEEK_END)
                            f.truncate()
                            f.write(b',\n')
                            new_d_json = json.dumps(d_json, indent=4)
                            f.write(bytes(new_d_json, 'utf-8'))
                            f.write(b'\n]')

                    elif descriptions[2].text[1:12] == 'Materials :':
                        craft = descriptions[1].text.lower()
                        reg = descriptions[3].text[9:].lower()
                        tri = descriptions[4].text[8:].lower()
                        
                        materials = descriptions[2].text[13:]
                        if materials.count(',') > 0:
                            materials_list = materials.split(',')
                            mat = [str(item.strip()).lower() for item in materials_list]
                        else:
                            print('\n\nthere is just one material\n')
                            mat = materials.strip().lower()
                        
                        with open("rugs-lemmatized-description.csv", mode='a', newline='') as f:
                            writer = csv.writer(f)
                            if materials.count(',') > 0:
                                writer.writerow([' '.join(mat) + ' ' + reg + ' ' + tri])
                            else:
                                writer.writerow([mat + ' ' + reg + ' ' + tri])

                        d_json = {'image': str(img), 'title': str(title), 'price': int(price), 'description': None, 'craft': str(craft), 'materials': mat, 'region': str(reg), 'tribe': str(tri)}
                        with open('rugs.json', 'rb+') as f:
                            f.seek(-1, os.SEEK_END)
                            f.truncate()
                            f.write(b',\n')
                            new_d_json = json.dumps(d_json, indent=4)
                            f.write(bytes(new_d_json, 'utf-8'))
                            f.write(b'\n]')
                    
                    else:
                        print('\n\nNo material found!!\n')


                except:
                    print(f"extraction of '{product_full_url}' data faild")

In [14]:
# scrape and save the data into "bert-rugs.json" and into "rugs-bert-description.csv" with bert

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# https://www.babakech.com/vintage-rugs/
# https://www.babakech.com/checkered-rugs/
# https://www.babakech.com/abstract-moroccan-rugs/
# https://www.babakech.com/abstract-moroccan-rugs?page=2
# https://www.babakech.com/abstract-moroccan-rugs?page=3
# https://www.babakech.com/abstract-moroccan-rugs?page=4
# https://www.babakech.com/kilim-moroccan-rugs/
# https://www.babakech.com/kilim-moroccan-rugs?page=2
# https://www.babakech.com/kilim-moroccan-rugs?page=3

request = rq.get(url='https://www.babakech.com/kilim-moroccan-rugs?page=3')
if request.status_code == 200:
    page = bs(request.content, 'html.parser')
    items = page.find_all(class_='product_details')

    for i in items:
        if i.find(class_='product-tag').text[:5].strip() == 'SKU:':
            product_url = i.find(class_='product-item-link').get('href')
            product_full_url = 'https://www.babakech.com' + product_url
            

            req = rq.get(url=product_full_url)
            if req.status_code == 200:
                soup = bs(req.content, 'html.parser')
                img = soup.find(class_='zoneimage fancybox').get('href')
                title = soup.find(class_='product_title').text[:-1].lower()

                price = soup.find(id='price-amount').text
                component = soup.find(class_='product_short_description')
                descriptions = component.find_all('p')
                try:
                    if descriptions[3].text[1:12] == 'Materials :':
                        desc = descriptions[0].text.lower().replace('\u00a0', ' ').replace('\u00e9', 'e').replace('\u2019', '\'').replace('\u2014', '-')
                        craft = descriptions[2].text.lower()
                        reg = descriptions[4].text[9:].lower()
                        tri = descriptions[5].text[8:].lower()
                        
                        materials = descriptions[3].text[13:]
                        if materials.count(',') > 0:
                            materials_list = materials.split(',')
                            mat = [str(item.strip()).lower() for item in materials_list]
                            mat = ' '.join(mat)
                        else:
                            print('\n\nthere is just one material\n')
                            mat = materials.strip().lower()
                        
                        
                        bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
                        model_bert = BertModel.from_pretrained('bert-base-uncased').to(device)
                        
                        # Tokenize and encode the description
                        encoded_desc = bert_tokenizer.encode(desc, add_special_tokens=True)
                        tokenized_desc = torch.tensor(encoded_desc).unsqueeze(0).to(device)
                        
                        # Get BERT embeddings
                        with torch.no_grad():
                            embedded_desc = model_bert(tokenized_desc)[0][0].mean(dim=0)


                        # Tokenize and encode the material
                        encoded_mat = bert_tokenizer.encode(mat, add_special_tokens=True)
                        tokenized_mat = torch.tensor(encoded_mat).unsqueeze(0).to(device)
                        
                        # Get BERT embeddings
                        with torch.no_grad():
                            embedded_mat = model_bert(tokenized_mat)[0][0].mean(dim=0)
                        
                        
                        # Tokenize and encode the region
                        encoded_reg = bert_tokenizer.encode(reg, add_special_tokens=True)
                        tokenized_reg = torch.tensor(encoded_reg).unsqueeze(0).to(device)
                        
                        # Get BERT embeddings
                        with torch.no_grad():
                            embedded_reg = model_bert(tokenized_reg)[0][0].mean(dim=0)
                        
                        
                        # Tokenize and encode tribe
                        encoded_tri = bert_tokenizer.encode(tri, add_special_tokens=True)
                        tokenized_tri = torch.tensor(encoded_tri).unsqueeze(0).to(device)
                        
                        # Get BERT embeddings
                        with torch.no_grad():
                            embedded_tri = model_bert(tokenized_tri)[0][0].mean(dim=0)
                        
                        description = embedded_desc + embedded_mat + embedded_reg + embedded_tri
                        
                        description_list = description.cpu().numpy().tolist()

                        with open("rugs-bert-description.csv", mode='a', newline='') as f:
                            writer = csv.writer(f)
                            writer.writerow(description_list)
                        
                        d_json = {'image': str(img), 'title': str(title), 'price': int(price), 'description': str(desc), 'craft': str(craft), 'materials': mat, 'region': str(reg), 'tribe': str(tri)}
                        with open('bert-rugs.json', 'rb+') as f:
                            f.seek(-1, os.SEEK_END)
                            f.truncate()
                            f.write(b',\n')
                            new_d_json = json.dumps(d_json, indent=4)
                            f.write(bytes(new_d_json, 'utf-8'))
                            f.write(b'\n]')

                    elif descriptions[2].text[1:12] == 'Materials :':
                        craft = descriptions[1].text.lower()
                        reg = descriptions[3].text[9:].lower()
                        tri = descriptions[4].text[8:].lower()
                        
                        materials = descriptions[2].text[13:]
                        if materials.count(',') > 0:
                            materials_list = materials.split(',')
                            mat = [str(item.strip()).lower() for item in materials_list]
                            mat = ' '.join(mat)
                        else:
                            print('\n\nthere is just one material\n')
                            mat = materials.strip().lower()
                        
                        
                        bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
                        model_bert = BertModel.from_pretrained('bert-base-uncased').to(device)


                        # Tokenize and encode the material
                        encoded_mat = bert_tokenizer.encode(mat, add_special_tokens=True)
                        tokenized_mat = torch.tensor(encoded_mat).unsqueeze(0).to(device)
                        
                        # Get BERT embeddings
                        with torch.no_grad():
                            embedded_mat = model_bert(tokenized_mat)[0][0].mean(dim=0)
                        
                        
                        # Tokenize and encode the region
                        encoded_reg = bert_tokenizer.encode(reg, add_special_tokens=True)
                        tokenized_reg = torch.tensor(encoded_reg).unsqueeze(0).to(device)
                        
                        # Get BERT embeddings
                        with torch.no_grad():
                            embedded_reg = model_bert(tokenized_reg)[0][0].mean(dim=0)
                        
                        
                        # Tokenize and encode tribe
                        encoded_tri = bert_tokenizer.encode(tri, add_special_tokens=True)
                        tokenized_tri = torch.tensor(encoded_tri).unsqueeze(0).to(device)
                        
                        # Get BERT embeddings
                        with torch.no_grad():
                            embedded_tri = model_bert(tokenized_tri)[0][0].mean(dim=0)
                        
                        description = embedded_mat + embedded_reg + embedded_tri
                        
                        description_list = description.cpu().numpy().tolist()

                        with open("rugs-bert-description.csv", mode='a', newline='') as f:
                            writer = csv.writer(f)
                            writer.writerow(description_list)

                        d_json = {'image': str(img), 'title': str(title), 'price': int(price), 'description': None, 'craft': str(craft), 'materials': mat, 'region': str(reg), 'tribe': str(tri)}
                        with open('bert-rugs.json', 'rb+') as f:
                            f.seek(-1, os.SEEK_END)
                            f.truncate()
                            f.write(b',\n')
                            new_d_json = json.dumps(d_json, indent=4)
                            f.write(bytes(new_d_json, 'utf-8'))
                            f.write(b'\n]')
                    
                    else:
                        print('\n\nNo material found!!\n')


                except:
                    print(f"extraction of '{product_full_url}' data faild")

In [ ]:
# read "rugs.json" data

# d_json = {'image': str(img), 'title': str(title), 'price': int(price), 'description': str(desc), 'craft': str(craft), 'materials': d[0][4], 'region': str(reg), 'tribe': str(tri)}
with open('rugs.json', 'r') as f:
    print(f.read())
    
    # for i, line in enumerate(f):
    #     if i == 111:
    #         print(line)
    #         break

In [ ]:
# read "rugs-lemmatized-description.csv" data

with open("rugs-lemmatized-description.csv", mode='r') as f:
    print(f.read())

In [ ]:
!pip uninstall neuspell

In [ ]:
import spacy

# Load the large English NLP model
nlp = spacy.load('en_core_web_lg')

# The text we want to examine
text = "dog cat banana"

# Parse the text with spaCy
# Our 'document' variable now contains a full parse of the text
doc = nlp(text)

# Using the parsed doc, find similarity between word tokens
for token1 in doc:
    for token2 in doc:
        print(token1.text, token2.text, token1.similarity(token2))
